# zilean Demo

In this demo, you can explore how to fetch data using the `TimelineCrawler` class. You then can explore how to manipulate data using the `SnapShots` class. 

To fetch data from the Riot Games API, you need your api key. It is free to get one if you have a Riot account, just visit the [Riot Developer Portal](https://developer.riotgames.com/) and log in. 

If you don't have a API Key, don't worry! We had prepared some data for you to play around. 

In [1]:
# Run this cell
from zilean import SnapShots, TimelineCrawler, read_api_key
import pandas as pd
import json

In [2]:
# !!DON'T RUN THIS CELL IF YOU DON'T HAVE AN API KEY!!

# Insert your api key here, replacing None
api_key = read_api_key(api_key=None)

In [3]:
# RUN THIS CELL IF YOU DON'T HAVE AN API KEY
if not api_key:
    FILE = "data.json"

## Using the TimelineCrawler

`TimelineCrawler` automatically help you crawl matches with your api key. You can customize the crawler to fetch matches based on your specification.

You can choose the **region** from these options:

    ['br1', 'eun1', 'euw1', 'jp1', 'kr', 'la1', 'la2', 'na1', 'oc1', 'ru', 'tr1']

You can choose the **tier** from these options:

    ['CHALLENGER','GRANDMASTER','MASTER','DIAMOND','PLATINUM','GOLD','SILVER','BRONZE','IRON'] 

You can choose the **queue** from these options:   

    ['RANKED_SOLO_5x5', 'RANKED_FLEX_SR', 'RANKED_FLEX_TT']

## Demo

In this demo, we will crawl 50 diamond ranked solo matches from the Korean server.

We can also specify the `cutoff` of our matches. We can only keep matches that lasted longer than a specific cutoff time. In this demo, the default value is 16 (in minutes).

Crawling 50 matches will usually take less than 1 minutes. 

_If you don't have a api key, the demo file will have 10 matches_.

In [4]:
# Using the api key
if api_key:
    crawler = TimelineCrawler(api_key, region="kr", tier="DIAMOND", queue="RANKED_SOLO_5x5")
    result = crawler.crawl(50, cutoff=16)
# No api key, load the local file
else:
    with open(FILE, "r") as f:
        result = json.load(f)

Crawling matches: 100%|██████████| 50/50 [00:37<00:00,  1.32it/s]
50it [00:00, 57.29it/s]


There are in total 50 crawled matches longer than 16 minutes.


Let's see the matchid of the first crawled match.

In [5]:
print("First match id:", result[0]['metadata']['matchId'])

First match id: KR_5995486857


## Using the SnapShots class

You can take a look at `result`, which is a list of `MatchTimelineDto`s. It is a very large and messy dictionary. We can ectract core player data of the matches by using the `SnapShots` class.

One of the most important feature of the `SnapShots` class is that you can specify which **frame** to look at. A **frame** is a snapshot of a match, in minutes. For example, the 15th frame is the game stats at the 15 minute mark of the game. With the `SnapShots` class, you can specify the frames you want to look at for all the matches you have crawled. 

In this demo, we will look at the **10 and 15** minute mark of every match.

In [6]:
snaps = SnapShots(result, frames=[10, 15])

We can see the statistics of the matches at 10 and 15 minutes using `.summary()`. For better visualization, we can feed the summary into a pandas DataFrame.

In [7]:
df = pd.DataFrame(snaps.summary())
print("Shape:", df.shape)
df.head(10)

Shape: (50, 102)


,level_0_frame10,timeEnemySpentControlled_0_frame10,totalGold_0_frame10,xp_0_frame10,totalDamageDone_0_frame10,totalDamageDoneToChampions_0_frame10,totalDamageTaken_0_frame10,creepScore_0_frame10,goldPorportion_0_frame10,xpPorportion_0_frame10,...,totalGold_4_frame15,xp_4_frame15,totalDamageDone_4_frame15,totalDamageDoneToChampions_4_frame15,totalDamageTaken_4_frame15,creepScore_4_frame15,goldPorportion_4_frame15,xpPorportion_4_frame15,matchId,win
0,0,-33579,948,-206,-1145,-830,1481,-23,0.035226,-0.005441,...,-232,-419,-17286,1727,-3873,0,-0.027849,-0.033486,KR_5995486857,False
1,1,-37707,-1094,1112,11304,-378,850,51,-0.041367,0.085176,...,-89,711,-25101,-1943,-1908,-71,-0.005988,0.023789,KR_5995471690,False
2,-1,-55346,-119,-333,-7565,-3474,165,-21,-0.021047,-0.028987,...,1301,91,7690,4882,-306,-9,0.029995,-0.010708,KR_5995404891,True
3,1,-120405,12,692,-10652,4065,-483,17,0.002268,0.050273,...,64,580,10734,2982,-124,3,0.015226,0.026878,KR_5995277927,True
4,1,-12091,739,697,6680,85,-218,1,0.030134,0.037032,...,272,-1048,31705,-1660,4758,101,0.000369,-0.030514,KR_5985259345,True
5,0,-108890,-1206,-434,-13181,-1780,1166,-25,-0.065087,-0.014957,...,1369,359,5840,-323,-683,16,0.051694,0.028447,KR_5984683095,True
6,1,22197,656,837,6731,-163,-383,17,0.052906,0.046747,...,-1343,-1067,-2036,-2955,46,2,-0.047502,-0.034713,KR_5984670506,False
7,1,-5714,1445,750,15192,1519,536,17,0.036249,0.023917,...,2919,2691,47480,6191,-1158,94,0.072139,0.090337,KR_5977763210,True
8,-1,-25503,164,-1255,-1936,-546,-2570,2,-0.024527,-0.112043,...,1388,3606,64971,3980,3556,125,0.022662,0.100318,KR_5977668471,True
9,2,41572,998,1567,13954,1950,-561,12,0.072681,0.088182,...,-641,-1119,3255,-753,809,16,0.009256,-0.009773,KR_5977624577,False


## Features

Each number represents the **difference** between the blue side and the red side. The column names might be very confusing at first, but you will understand it if you know how the columns are named. 

The columns names follow this pattern: _feature\_lane#\_frame#_. 

For example: _level\_0\_frame10_ represents the **difference in level between the 0th lane (TOP lane) player at the 10 minute mark**. For conversion of the lane code, please refer:

    {0:"TOP", 1:"JUG", 2:"MID", 3:"BOT", 4:"SUP"}

## What's Next?

Data from all the matches are now in an organized 2 dimension tabular format. From here, we can feed the data to your favorite machine learning models. 

You can do more manipulation using the `SnapShots` class. For example, if you are familiar with League of Legends, you can sepcify the **lanes** you are interested in

In [8]:
top_and_mid = snaps.get_lanes(["TOP", "MID"], per_frame=False)
pd.DataFrame(top_and_mid).head(10)

,level_0_frame10,timeEnemySpentControlled_0_frame10,totalGold_0_frame10,xp_0_frame10,totalDamageDone_0_frame10,totalDamageDoneToChampions_0_frame10,totalDamageTaken_0_frame10,creepScore_0_frame10,goldPorportion_0_frame10,xpPorportion_0_frame10,...,totalGold_2_frame15,xp_2_frame15,totalDamageDone_2_frame15,totalDamageDoneToChampions_2_frame15,totalDamageTaken_2_frame15,creepScore_2_frame15,goldPorportion_2_frame15,xpPorportion_2_frame15,matchId,win
0,0,-33579,948,-206,-1145,-830,1481,-23,0.035226,-0.005441,...,-910,-546,-1551,-2963,2038,10,-0.047583,-0.030699,KR_5995486857,False
1,1,-37707,-1094,1112,11304,-378,850,51,-0.041367,0.085176,...,320,-1879,-3987,-407,3584,-13,0.007383,-0.059235,KR_5995471690,False
2,-1,-55346,-119,-333,-7565,-3474,165,-21,-0.021047,-0.028987,...,1661,815,2964,1568,-855,7,0.033472,0.006696,KR_5995404891,True
3,1,-120405,12,692,-10652,4065,-483,17,0.002268,0.050273,...,-346,-715,8087,983,-1748,-3,0.000320,-0.011836,KR_5995277927,True
4,1,-12091,739,697,6680,85,-218,1,0.030134,0.037032,...,967,470,445,5001,365,-16,0.022678,0.022568,KR_5985259345,True
5,0,-108890,-1206,-434,-13181,-1780,1166,-25,-0.065087,-0.014957,...,1030,-1089,27587,-3502,-2738,15,0.038531,-0.012440,KR_5984683095,True
6,1,22197,656,837,6731,-163,-383,17,0.052906,0.046747,...,-116,-596,-7373,-3912,3791,-19,0.002752,-0.017316,KR_5984670506,False
7,1,-5714,1445,750,15192,1519,536,17,0.036249,0.023917,...,1699,385,3597,455,-2742,34,0.031891,0.016061,KR_5977763210,True
8,-1,-25503,164,-1255,-1936,-546,-2570,2,-0.024527,-0.112043,...,1251,1539,16564,2474,-3179,42,0.020229,0.030919,KR_5977668471,True
9,2,41572,998,1567,13954,1950,-561,12,0.072681,0.088182,...,-2657,-2448,-23283,-6035,2730,-32,-0.057117,-0.039703,KR_5977624577,False
